In [1]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [2]:
def build_prompt(system_message: str, user_message: str):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

# Basic chatbot function

In [3]:
system_message =  "You are a helpful assistant."
user_message = "How do I reset a Zoom Room tablet?"

messages = build_prompt(system_message, user_message)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)
print(response.choices[0].message.content)


To reset a Zoom Room tablet, you'll need to perform a factory reset. Here's a general guide on how to do this, but be aware that the steps can vary slightly depending on the brand and model of the tablet you are using (such as an iPad, Android-based tablet, etc.). Here are the steps for some common devices:

### For Apple iPad:
1. **Back Up Your Data** (if needed): Before resetting, ensure any necessary data is backed up.
2. **Open Settings** on the iPad.
3. Go to **General**.
4. Scroll down and tap **Transfer or Reset iPad**.
5. Tap **Erase All Content and Settings**.
6. Enter your passcode if required, and confirm that you want to erase the device.
7. The iPad will restart and begin the reset process.

### For Android Tablets:
1. **Back Up Your Data** (if needed): Ensure important data is backed up as a reset will erase all data.
2. **Open Settings** on the tablet.
3. Navigate to **System** (you might need to expand the menu depending on the tablet's user interface).
4. Tap on **Rese

# Using system instructions 

In [4]:
system_message = "You are an internal IT helpdesk bot. Always ask for the room name and device ID before giving troubleshooting advice."
user_message = "The screen in my conference room isn't turning on."

messages = build_prompt(system_message, user_message)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)
print(response.choices[0].message.content)


I can help with that issue! Could you please provide me with the room name and the device ID of the screen?


In [5]:
user_message = "Sure, I am in room 210 using screen A15."

messages = build_prompt(system_message, user_message)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)
print(response.choices[0].message.content)

Thank you for providing the room name and device ID. How can I assist you today with screen A15 in room 210?


### The OpenAI API is stateless, so need to append the entire conversation to create a continuous thread.

In [6]:
class ChatSession:
    def __init__(self, system_prompt: str, model: str = "gpt-4o", api_key: str = None):
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.messages = [{"role": "system", "content": system_prompt}]
    
    def send(self, user_input: str, temperature: float = 0.7) -> str:
        '''iterative chat send'''
        self.messages.append({"role": "user", "content": user_input})
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=temperature
        )
        
        assistant_reply = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_reply})
        return assistant_reply
    
    def reset(self, new_system_prompt: str = None):
        if new_system_prompt:
            self.messages = [{"role": "system", "content": new_system_prompt}]
        else:
            self.messages = self.messages[:1]  # keep original system prompt only
    
    def show_history(self):
        return self.messages

In [19]:
session = ChatSession(
    system_prompt="You are an internal IT helpdesk bot. Always ask for the room name and device ID before giving troubleshooting advice.",
    api_key=os.environ["OPENAI_API_KEY"]
)

reply1 = session.send("The screen in my conference room isn't turning on.")
print("Assistant:", reply1)

reply2 = session.send("It's Room 3A. The device ID is ZR3A-0093.")
print("Assistant:", reply2)

Assistant: I'm here to help with that! Could you please provide me with the room name and the device ID of the screen you're having trouble with?
Assistant: Thank you for the information. Let's try a few troubleshooting steps for the screen in Room 3A with device ID ZR3A-0093:

1. **Power Connection**: Ensure that the screen is properly plugged into a power outlet. Check if the power cable is securely connected to both the screen and the outlet.

2. **Power Button**: Press the power button on the screen to see if it turns on. Sometimes, the screen may be in standby mode.

3. **Remote Control**: If you are using a remote control, check the batteries and try turning on the screen using the remote.

4. **Input Source**: Verify that the screen is set to the correct input source. If there are multiple connected devices, ensure that the input matches the device you are trying to use.

5. **Cable Connections**: Check all the cable connections between the screen and any connected devices. Make

# Integrating RAG

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

docs = [
    "If a Zoom Room tablet is unresponsive, hold the power button for 10 seconds to restart it.",
    "To reset a Cisco conferencing camera, unplug it for 30 seconds then plug it back in.",
    "To file a support ticket, go to go/support and click 'Report AV Issue'."
]

# vector store creates an embedded representation of the docs
vectorstore = FAISS.from_texts(docs, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

# first use the "retriever" to match embedded docs to the user message
user_message = "How can I fix a frozen Zoom Room screen?"
context_docs = retriever.invoke(user_message)
context = "\n".join(doc.page_content for doc in context_docs)

# then feed this matched documentation to provide to the chatbot as additional context in the system message
system_message = f"You are a helpdesk assistant. Use the following context to answer user questions:\n\n{context}"
session = ChatSession(system_message, api_key=os.environ["OPENAI_API_KEY"])
response = session.send(user_message)
print("Assistant:", response)


Assistant: If the Zoom Room tablet is unresponsive or frozen, you can fix it by holding the power button for 10 seconds to restart it.


In [14]:
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# load and format RAG docs
loader = DirectoryLoader("RAG_docs", glob="**/*.txt", loader_cls=TextLoader)
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)

# create vector store of doc embeddings
vectorstore = FAISS.from_documents(chunks, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

# retrieve relevant documentation
query = "Where should I go to report a problem with the AV system?"
context_docs = retriever.get_relevant_documents(query)
context = "\n".join(doc.page_content for doc in context_docs)

# start chatbot session with query and retrieved context
system_msg = f"You are a company helpdesk bot. Use the following context to answer the user's question accurately:\n\n{context}"
session = ChatSession(system_msg, api_key=os.environ["OPENAI_API_KEY"])
response = session.send(query)
print("Assistant:", response)


Assistant: To report a problem with the AV system, go to go/support and click "New Request". Then, choose the category "AV or Meeting Rooms" and provide the room name, device ID, and a short description of the issue. This will ensure your ticket is routed to the AV team automatically.


# Generate Ticket

In [16]:
def generate_keywords_with_llm(query: str, reply: str, api_key: str, model: str = "gpt-4o") -> list[str]:    
    keyword_prompt = (
        "You are an AI assistant that extracts relevant keywords from helpdesk queries and responses.\n"
        "Return a comma-separated list of 3 to 5 keywords.\n\n"
        f"Query: {query}\n\n"
        f"Response: {reply}\n\n"
        "Keywords:"
    )

    session = ChatSession(system_prompt="You extract helpful keywords from text.", model=model, api_key=api_key)
    keyword_string = session.send(keyword_prompt)
    return [kw.strip() for kw in keyword_string.split(",") if kw.strip()]


def create_ticket(user_query: str, assistant_reply: str, context_snippets: list[str], keywords: list[str]) -> dict:
    return {
        "summary": user_query,
        "response": assistant_reply,
        "context_used": context_snippets,
        "status": "open",
        "created_by": "RAG-bot",
        "category": "AV Support",
        "keywords": keywords
    }

In [18]:
query = "Where should I go to report a problem with the AV system?"
context_docs = retriever.invoke(query)
context_snippets = [doc.page_content for doc in context_docs]
context = "\n".join(context_snippets)

system_msg = f"You are a company helpdesk bot. Use the following context to answer the user's question accurately:\n\n{context}"
session = ChatSession(system_msg, api_key=os.environ["OPENAI_API_KEY"])
assistant_reply = session.send(query)

#generate keywords
keywords = generate_keywords_with_llm(query, assistant_reply, api_key=os.environ["OPENAI_API_KEY"])

ticket = create_ticket(query, assistant_reply, context, keywords)

# Show results
print("🧠 Assistant:", assistant_reply)
print("\n📝 Ticket Created:")
for key, value in ticket.items():
    print(f"{key.upper()}:\n{value}\n")


🧠 Assistant: To report a problem with the AV system, you should file an internal support ticket by going to go/support. Then, click "New Request" and choose the category "AV or Meeting Rooms". Make sure to provide the room name, device ID, and a short description of the issue to ensure the ticket is routed correctly to the AV team.

📝 Ticket Created:
SUMMARY:
Where should I go to report a problem with the AV system?

RESPONSE:
To report a problem with the AV system, you should file an internal support ticket by going to go/support. Then, click "New Request" and choose the category "AV or Meeting Rooms". Make sure to provide the room name, device ID, and a short description of the issue to ensure the ticket is routed correctly to the AV team.

CONTEXT_USED:
To file an internal support ticket:

1. Go to go/support.
2. Click "New Request".
3. Choose the category "AV or Meeting Rooms".
4. Provide room name, device ID, and a short description of the issue.

Tickets are routed to the AV team

# More complete example

#### Includes:
- Keyword logic to recognize different types of user queries
- Ongoing RAG capability
- End of conversation behavior

### Other possible additions:
- LLM logic to route queries (RAG, session end, live human?)
- Ability to submit open ticket
- Guardrails
- ??

In [51]:
from datetime import datetime
from openai import OpenAI
from langchain.schema import BaseRetriever

class ChatSession:
    def __init__(self, system_prompt: str, retriever: BaseRetriever = None, model: str = "gpt-4o", api_key: str = None):
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.api_key = api_key
        self.retriever = retriever
        self.is_active = True
        self.start_time = datetime.now()
        self.messages = [{"role": "system", "content": system_prompt}]
        self.retrieval_log = []  # store RAG context per turn if used

    def handle_input(self, user_input: str, retriever=None, temperature: float = 0.7) -> str:
        '''logic for using RAG and ending the session--hard coded'''
        input_lower = user_input.lower()

        self.messages.append({"role": "user", "content": user_input})
        
        # check for language indicating the end of the session
        if any(phrase in input_lower for phrase in ["that's all", "end session", "you can close this", "goodbye", "bye", "that's it"]):
            ticket = self.end()
            print("📄 Ticket Generated:\n")
            for k, v in ticket.items():
                print(f"{k.upper()}:\n{v}\n")
            return "Session ended and ticket created."
    
        # check for language indicating that RAG should be used
        use_rag = self.retriever is not None and any(kw in input_lower for kw in ["lookup", "look up", "reference", "context", "support", "reset", "fix", "how do i"])
        
        if use_rag:
            return self.send_with_rag(user_input, temperature=temperature)
        else:
            return self.send(user_input, temperature=temperature)


    def send(self, user_input: str, temperature: float = 0.7) -> str:
        '''basic iterative chat'''
        if not self.is_active:
            raise RuntimeError("This session has ended.")
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=temperature
        )
        reply = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": reply})
        return reply

    def send_with_rag(self, user_query: str, temperature: float = 0.7, include_context: bool = True) -> str:
        '''iterative chat with RAG lookup'''
        if not self.is_active:
            raise RuntimeError("This session has ended.")

        if include_context:
            context_docs = self.retriever.invoke(user_query)
            context = "\n".join(doc.page_content for doc in context_docs)
            self.retrieval_log.append({"query": user_query, "context": context})
            modified_query = f"Context:\n{context}\n\nQuestion:\n{user_query}"
        else:
            modified_query = user_query

        return self.send(modified_query, temperature=temperature)

    def reset(self, new_system_prompt: str = None):
        if new_system_prompt:
            self.messages = [{"role": "system", "content": new_system_prompt}]
        else:
            self.messages = self.messages[:1]
        self.is_active = True

    def show_history(self):
        return self.messages

    def end(self, generate_ticket: bool = True, use_llm_keywords: bool = True) -> dict:
        '''end session and generate ticket'''
        self.is_active = False
    
        if not generate_ticket:
            return {}
    
        # combine entire conversation history
        conversation_pairs = [
            (self.messages[i]["content"], self.messages[i+1]["content"])
            for i in range(1, len(self.messages)-1, 2)
            if self.messages[i]["role"] == "user" and self.messages[i+1]["role"] == "assistant"
        ]
    
        # use initial query and response for "headline" summary
        user_query = conversation_pairs[0][0] if conversation_pairs else ""
        assistant_reply = conversation_pairs[0][1] if conversation_pairs else ""
    
        # include all RAG context
        context_snippets = [r["context"] for r in self.retrieval_log]
    
        # generate keywords
        if use_llm_keywords:
            keywords = generate_keywords_with_llm(user_query, assistant_reply, api_key=self.api_key, model=self.model)
        else:
            keywords = extract_keywords(user_query + " " + assistant_reply, KNOWN_KEYWORDS)
    
        ticket = {
            "headline": {
                "question": user_query,
                "answer": assistant_reply,
            },
            "keywords": keywords,
            "timestamp": datetime.now().isoformat(),
            "turns": len([m for m in self.messages if m["role"] == "user"]),
            "full_conversation": self.messages,
            "rag_context_log": self.retrieval_log
        }
        return ticket



In [52]:
session = ChatSession(
    system_prompt="You are an AV helpdesk bot. Use context when needed. Ask the user if they need anything else at the end of each response.",
    api_key=os.environ["OPENAI_API_KEY"],
    retriever=retriever
)

# Simulate dialogue
print("Assistant:", session.handle_input("How do I reset a Zoom Room controller?"))
print("Assistant:", session.handle_input("Can you look up what to do if that doesn't fix the black screen?"))
print(session.handle_input("That's all, thank you."))

Assistant: To reset a Zoom Room controller, follow these steps:

1. **Open the Zoom Room Controller App:** On the device running the Zoom Room controller, open the Zoom Room Controller app.

2. **Access Settings:** Tap the gear icon or the settings button in the app. This is usually located in the bottom corner of the screen.

3. **Enter Admin Settings:** You may be prompted to enter an admin password. Enter the password to access the admin settings.

4. **Reset Controller:** Look for an option labeled "Reset Controller" or similar. This option might be under "Advanced" or "About" sections, depending on the version. Tap on it.

5. **Confirm Reset:** You will be asked to confirm the reset action. Confirm that you want to reset the controller.

6. **Reconfigure:** After the reset, you will need to reconfigure the Zoom Room controller. This involves connecting it to the Zoom Room and entering any necessary settings.

Please note that the exact steps may vary slightly depending on the vers